In [134]:
import numpy as np
import time
import psi4
from IPython.display import HTML
import random


In [135]:
# fonction calcul X=Us^-1/2U^dagger à partir de matrice overlap S, une matrice overlap n'a que des
# valeurs propres strictement positives (définie positive), donc pas de problèmes pour calculer s=np.diag(1/np.sqrt(evalues))

    
def verif_X_matrix(X,S):
    Xh=np.conjugate(X.T)
    n=len(X)
    verif=np.matmul(Xh,np.matmul(S,X))
    if (abs(verif-np.identity(n))<10**-14).all():
        return True
    else:
        return(False)

def X_matrix(S):
    evalues,U= np.linalg.eigh(S) 
    s=np.diag(1/np.sqrt(evalues))#matric diagonale avec l'inverse des racines des valeurs propres

    X=np.matmul(np.matmul(U,s),np.conjugate(U.T)) #X=Us^(-1/2)U*
    if verif_X_matrix(X,S):
        return(X)
    else: 
        return('error')


In [136]:
#Calcul de F a l'aide de la matrice densité F= H + G
def G_matrix(P,v):
    n=len(P)
    G=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            G_ij=0
            for k in range(n):
                for l in range(n):
                    G_ij+=P[k,l]*(v[i][j][k][l]-0.5*v[i][l][k][j])
            G[i,j]=G_ij
    return(G)
    
#fonction calcul  F'=X*FX

def new_fock_matrix(F,X):
    FX=np.matmul(F,X)
    return(np.matmul(np.conjugate(X.T),FX))

In [137]:
#calcul matrice C des coefs

def coef_matrix(f,X):
    energy,c=np.linalg.eigh(f)
    C=np.matmul(X,c)
    return(C)

In [138]:
#calcul nouvelle matrice densité
def new_density_matrix(C):
    return(2*np.matmul(C,np.conjugate(C.T)))


In [139]:
#convergence de l'algo avec le commutateur
def convergence(F,P,S,lim):
    FPS=np.matmul(F,np.matmul(P,S))
    SPF=np.matmul(S,np.matmul(P,F))
    if (abs(FPS-SPF)<=lim).all():
        return True
    else:
        return False
    

    

In [140]:
psi4.set_memory('500 MB')

mol = psi4.geometry("""
H  0.0  0.000000  0.0
H  0.75  0.000000  0.0

""")


geo_unit = psi4.core.GeometryUnits(0) # Angstrom:0 , bohr:1
null=psi4.core.Molecule.set_units(mol,geo_unit)
null=psi4.core.Molecule.set_molecular_charge(mol,0)
null=psi4.core.Molecule.set_multiplicity(mol,1)


In [141]:
wf = psi4.core.Wavefunction.build(mol,basis='sto-3g')
mints = psi4.core.MintsHelper(wf.basisset())


# Overlap
S = np.array(mints.ao_overlap())
# Intégrales biélectroniques (ERI: Electronic Repulsion Integrals)
v = np.asarray(mints.ao_eri())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
# intégrale monoéléctronique
H=T+V


In [142]:
X=X_matrix(S)
P=np.zeros_like(H)
print(P)
lim=10**(-14)

[[0. 0.]
 [0. 0.]]


In [147]:
n=50
print()
for i in range(n):
    G=G_matrix(P,v)
    F=H+G
    f=new_fock_matrix(F,X)
    C=coef_matrix(f,X)
    P=new_density_matrix(C) 
    
print(0.5*np.trace(np.matmul(P,H+F)))
    


0.1959141897058292
